### This program is divided into the following sections:
1. **Feature Extraction** (all the functions I need for feature extraction)
2. **Preprocessing** (Making the feature set and count dictionary)
3. **Code for my multinomial NB** (Functions for implementing MNB)
4. **Using inbuilt MNB** (Using sklearns MNB)
5. **Using my own MNB**
6. **Classification Report** (For both classifiers)
7. **Comparitive result** (Comparing on the basis of precision, recall and f1-score)

Note : The whole program may take upto 5 mins to run (i7 gen processor, 8GB)

# FEATURE EXTRACTION

### Fetching Stop-words

> Importing NLTK (Natural Language Toolkit)

In [1]:
import nltk
from nltk.corpus import stopwords
import re
from os import listdir

> Handling stopwords

In [2]:

# Downloading stopwords
nltk.download('stopwords')

# Loading English stopwords from inbuilt libraries
stop_words = list(stopwords.words('english'))

# Making my own list of stopwords, most of these are from metadata
my_stopwords = ['one', 'two', 'three', 'four', 'five', 'six',' seven', 'eight', 'nine', 'would', 'like','may',
                'without','get','could','might', 'edu','article','writes','people','think','thinks','write',
                'com','use','used', 'gatech', 'europa','cmu','apr','sci','srv','path','organisation','message',
               'id','sender','newgroups','gmt','date','subject','also']

# Appending two lists of stopwords to get a final list of all stopwords
stop_words = stop_words+ my_stopwords
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /home/satan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Utility functions

> fetch_labelled_paths: Gets a complete list of paths-class pairs in the given main folder

In [11]:
# Making a table (dictionary) containing all the paths of txt files (keys) in the given main folder (main_path), labelled 
# with the class (value)
def fetch_labelled_paths(main_path):    
    paths_data_labelled = dict()
    for cls in listdir(main_path):
        cls_path = main_path+'/'+cls
        for txt in listdir(cls_path):
            txt_path = cls_path + '/' + txt
            paths_data_labelled[txt_path] = cls
    return paths_data_labelled

In [12]:
dirs = fetch_labelled_paths('20_newsgroups')
dirs

{'20_newsgroups/comp.sys.mac.hardware/52005': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52202': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52223': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/51819': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52016': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52340': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/51705': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52032': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52095': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52255': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/50493': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52233': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/51874': 'comp.sys.mac.hardware',
 '20_newsgroups/comp.sys.mac.hardware/52197': 'comp.sys.mac.hardware',
 '20_n

> Freq: 
Gets the frequency of words in the file given by path (file_name) and adds it to the common class dictionary (di)

In [13]:
def freq(di, file_name):
    # Open file handle using 8-bit unicode encoding 
    handle = open(file_name, 'r', encoding = 'utf-8', errors="surrogateescape")
    
    # Get all the lines of the txt file and remove as much metadata as possible
    lines = handle.readlines()
  #  lines = remove_metadata(lines)
    
    # Loop through each line at a time
    # Eliminate every non alphabet character by substituting it with space character
    # Split line to obtain list of words
    # Update word and its frequency in the dictionary of the class
    # Ignore all the stopwords
    for line in lines:
        line = re.sub("[^a-zA-Z ]"," ",line)
        line = line.split(" ")
        for wrd in line:
            wrd = wrd.lower()
            if (not(wrd in stop_words) and (len(wrd) > 2)):
                if wrd in di:
                    di[wrd] = di[wrd] + 1
                else:
                    di[wrd] = 1
                    
    # Good habit. Close the file once processed 
    handle.close()
    return di

# PREPROCESSING

The main task of this section is to find the features, ie, the most common words from each class and merge them to form a single feature set.

> Get the list of classes (20 Types of doccuments)

In [14]:
# Import operating system module for listdir() function
from os import listdir

# Soecify the path containing all the class folders for the data
path = "20_newsgroups"

# Get the list of classes by listing out the directories in the specified path 
classes = [f for f in listdir(path)]

# To the stopwords, include the list of name as they may occur in the file headers pretty often
stop_words = stop_words+classes

# Print out classes
classes

['comp.sys.mac.hardware',
 'comp.graphics',
 'talk.politics.guns',
 'talk.politics.misc',
 'sci.crypt',
 'alt.atheism',
 'rec.motorcycles',
 'comp.os.ms-windows.misc',
 'talk.religion.misc',
 'rec.sport.hockey',
 'sci.electronics',
 'misc.forsale',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.autos',
 'talk.politics.mideast',
 'soc.religion.christian',
 'sci.space',
 'comp.sys.ibm.pc.hardware',
 'sci.med']

> Create a word_count (dictionary with all the vocabularies + frequencies of all classes) 

In [52]:
from operator import itemgetter

# Initialise k, the number of words with highest frequency per class 
k = 2000

# Initialise the common class-vocabulary dictionary as word_count
word_count = dict()

# Iterate through all classes
# Get the path to the folders of those classes as subpath
# List out ALL the paths to the text doccuments inside each class
# For every text file, update the class dictionary(vocabulary) using the freq function
# Sort out the class dictionary/vocabulary on the basis of value (word frequency)
# Trim the vocabulary to k words with highest vocabulary
# Add the class vocaulary to the word_count (dictionary for ALL classes)

for c in classes:
    subpath = path +'/' +c
    d = [(subpath+'/'+d)  for d in listdir(subpath)]
    di = dict()
    for txt_path in d:
        di = freq(di, txt_path)
        if '' in di:
            del(di[''])
    
    di = (sorted(di.items(), key=itemgetter(1)))
    di = di[-k:]
    di = dict(di)
    word_count[c] = di
    print(c)

comp.sys.mac.hardware
comp.graphics
talk.politics.guns
talk.politics.misc
sci.crypt
alt.atheism
rec.motorcycles
comp.os.ms-windows.misc
talk.religion.misc
rec.sport.hockey
sci.electronics
misc.forsale
comp.windows.x
rec.sport.baseball
rec.autos
talk.politics.mideast
soc.religion.christian
sci.space
comp.sys.ibm.pc.hardware
sci.med


In [53]:
di

{'lesions': 18,
 'profession': 18,
 'ent': 18,
 'nutrient': 18,
 'towards': 18,
 'elaine': 18,
 'roundup': 18,
 'crhc': 18,
 'mention': 18,
 'acupuncture': 18,
 'avoiding': 18,
 'harmful': 18,
 'comment': 18,
 'fred': 18,
 'huey': 18,
 'equally': 18,
 'trained': 18,
 'karl': 18,
 'clearly': 18,
 'ibd': 18,
 'votes': 18,
 'eric': 18,
 'douglas': 18,
 'mhollowa': 18,
 'umn': 18,
 'lbl': 18,
 'speaking': 18,
 'ontario': 18,
 'archive': 18,
 'wonderful': 18,
 'ringing': 18,
 'european': 18,
 'approval': 18,
 'art': 18,
 'understood': 18,
 'personally': 18,
 'bmdelane': 18,
 'cruzio': 18,
 'gone': 18,
 'relationship': 18,
 'finally': 18,
 'practitioners': 18,
 'rebound': 18,
 'decline': 18,
 'surprising': 18,
 'grams': 18,
 'metabolic': 18,
 'anywhere': 18,
 'chapter': 18,
 'rosemail': 18,
 'noise': 18,
 'west': 18,
 'dealing': 18,
 'followed': 18,
 'separate': 18,
 'kit': 18,
 'stevens': 18,
 'region': 18,
 'lundby': 18,
 'spool': 18,
 'dizziness': 18,
 'candidiasis': 18,
 'israel': 18,
 '

### Creating the 2D feature array
> Having made the dictionary of frequent words of all classes, we need to merge all words to get a feature set.

In [54]:
# Initialise an empty list
final_list = list()

# To the empty list, add all the words from all classes vocabularies
for x in word_count:
    final_list = final_list + list(word_count[x].keys())
    
# Among all the words, keep only unique words for the feature set, i.e, discard 
# repeated words for the final feature set
feature_words = list(set(final_list))   

In [55]:
#FINAL SET OF FEATURES
print(feature_words)

['ati', 'experiences', 'sheikh', 'null', 'board', 'unbreakable', 'struct', 'pipex', 'samples', 'kohlhepp', 'richk', 'mcgill', 'bitz', 'sweep', 'parameter', 'panthers', 'modifications', 'gentiles', 'gfcis', 'thorne', 'welfare', 'tests', 'comprehensive', 'infallible', 'ferrari', 'vgalogo', 'rubber', 'administrators', 'puc', 'pname', 'teach', 'winqvt', 'phds', 'karina', 'ceremonies', 'yeltsin', 'otherwise', 'fanaticism', 'hut', 'named', 'iti', 'sunfish', 'ames', 'libertarian', 'voted', 'bar', 'sabre', 'royals', 'joke', 'activity', 'calm', 'almanac', 'limitation', 'bring', 'ulan', 'damico', 'biological', 'cassini', 'fire', 'chevron', 'generation', 'dare', 'sun', 'mother', 'kennedy', 'macalstr', 'holos', 'bless', 'rings', 'nyu', 'colormaps', 'associated', 'imo', 'pregnency', 'elevated', 'interpolation', 'addr', 'decrypt', 'nords', 'amtower', 'distance', 'mfrhein', 'night', 'advertised', 'examples', 'thug', 'blanket', 'cas', 'ferch', 'xoutput', 'received', 'compile', 'springfield', 'fanatism

> Creating the actual list of lists that is to be used for the fetching words counts

# CODE FOR MNB

> Importing relevant modules

In [56]:
import numpy as np

> **calculate_probability**: takes a single input point at a time and finds the probability of it belonging to the class curr_class.

In [57]:
def calculate_probabilty(x,curr_class):
    # Find the probability of the class being curr_class
    class_probability = np.log(count[curr_class]['files_count']) - np.log(len(count))
    
    # Initialise return value (op) as class_probability
    op = class_probability
    
    # For all words in the feature set, check if the word is present in the doccument, ie,
    # its frequency is greater than one. If it is present, then it will contribute to the
    # P(w = wi/c = ci) part of the formula.
    for i in range(len(x)):
        if x[i] > 0:
            word = feature_words[i]
            num = count[curr_class][word] + 1
            den = count[curr_class]['total_word_count'] + len(count[curr_class])-2
            op = op + np.log(num) - np.log(den)
            
    # Return op
    return op    

> **predict_single**: predicts a single data point at a time.

In [58]:
def predict_single(x):
    # Inititialise the initial guess as -1 
    ind = -1
    
    # Initialise max probability as a large negative number
    maxP = -1000000
    
    # Iterate over all classes, and find the one with the maximum probability
    for i in classes:
        curr_class_prob = calculate_probabilty(x,i)
        if curr_class_prob > maxP:
            maxP = curr_class_prob
            ind = classes.index(i)
            
    # Return the index of that class        
    return ind

> **predict**: return the array of output predicted

In [59]:
def predict(X):
    # Get the number of test points
    m = (X.shape)[0]
    
    # Initialise  Y_pred as an array of 0s
    Y_pred = np.zeros(m, dtype = 'int8')
    
    # For all input test points, get the prediction and update Y_pred
    for i in range(m):
        Y_pred[i] = predict_single(X[i,:])
        
    # Return Y_pred
    return Y_pred

> **create_count_dictionary**: main dictionary of counts used in finding the probabilities.<br>It will have the following strucuture:
* count will have a list of classes at the primary level.
>> * Each class will have a dictionary of words, where the key represents the word and the value is the total number of times it has occured in ALL the files of that class 
>> * Each class will also have two extra entries: total_files (number of doccuments of this class) and total_count(total number of words in the feature set occrung in all files of this class) 
* count will also have an entry 'total_count', which is the total number of feature words in all classes

In [60]:
def create_count_dictionary(X_train, Y_train):
    # Initialise the count as an empty dictionary
    count = dict()
    
    # Add all the classes to our dictionary
    for cls in classes:
        count[cls] = dict()
        total = 0
        for wrd in feature_words:
            if wrd in word_count[cls]:
                count[cls][wrd] = word_count[cls][wrd]
            else:
                count[cls][wrd] = 0
            total = total + count[cls][wrd]
        count[cls]['total_word_count'] = total
        count[cls]['files_count'] = sum(Y_train==classes.index(cls))
    count['total_data'] = len(X_train)
    
    return count

> **create_X_Y:**
* Take input as the path to the directory/folder containing subdirectories/subfolders with txt files.
*  Returns X, Y. 
>>X: The input feature matrix where the features(feature_words) are defined earlier.  
Y: Label corresponding to the type of doccument/class.

In [61]:
def create_X_Y(path):
    # Initialise X as a dictionary to add path-features 
    paths = fetch_labelled_paths(path)
    
    # Dimensions of our feature matrix. Rows = No of docs, cols = features
    rows = len(paths)
    cols = len(feature_words)
    
    # Initialising X and Y
    X = np.zeros([rows, cols], dtype = 'int64')
    Y = np.zeros(rows, dtype = 'int64')
    row = 0
    
    # Parsing through all txt files
    for txt in paths:
        di = dict()
        
        # Getting words freq of each txt files
        di = freq(di, txt)
        for wrd in feature_words:
            if wrd in di:
                X[row][feature_words.index(wrd)] = di[wrd]
                
        # Get the type of doccument. Say, path = '20_newsgroups/comp.graphics/8673', then split 
        # using '/' as delimiter and get the middle element as the doccument type
        doc_type = txt.split('/')[1]
        
        # Store the class in umeric form instead of strings for convenience
        Y[row] = classes.index(doc_type)
        row = row+1
    return X,Y

# USING INBUILT MULTINOMIAL NAIVE BAYES

In [62]:
# Convert the training text files into feature vectors
X_train,Y_train = create_X_Y('20_newsgroups')

In [63]:
# Convert the testing test files into feature vectors
X_test, Y_test = create_X_Y('mini_newsgroups')

In [64]:
X_test.shape, X_train.shape

((2000, 11790), (19997, 11790))

In [65]:
# Import the model
from sklearn.naive_bayes import MultinomialNB

In [66]:
# Initialise the classifier
clf = MultinomialNB()

In [67]:
# Fit the classifier using training data
clf.fit(X_train,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [68]:
# Predict using this classifier
Y_pred_inbuilt = clf.predict(X_test)

# USING MY OWN MNB CODE

In [69]:
count = create_count_dictionary(X_train, Y_train)

In [70]:
Y_pred_my = predict(X_test)

# CLASSIFICATION REPORT

In [71]:
from sklearn.metrics import classification_report, confusion_matrix

In [72]:
print(classification_report(Y_test, Y_pred_inbuilt))
print(confusion_matrix(Y_test, Y_pred_inbuilt))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       100
           1       0.88      0.92      0.90       100
           2       0.88      0.95      0.91       100
           3       0.87      0.76      0.81       100
           4       0.99      0.98      0.98       100
           5       0.83      0.88      0.85       100
           6       0.97      0.99      0.98       100
           7       0.96      0.88      0.92       100
           8       0.75      0.71      0.73       100
           9       0.99      0.97      0.98       100
          10       0.95      0.93      0.94       100
          11       0.90      0.97      0.93       100
          12       0.97      0.90      0.93       100
          13       0.99      0.98      0.98       100
          14       1.00      0.99      0.99       100
          15       0.96      0.98      0.97       100
          16       0.99      0.99      0.99       100
          17       0.99    

In [73]:
print(classification_report(Y_test, Y_pred_my))
print(confusion_matrix(Y_test, Y_pred_my))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       100
           1       0.88      0.88      0.88       100
           2       0.83      0.92      0.87       100
           3       0.82      0.77      0.79       100
           4       0.98      0.96      0.97       100
           5       0.80      0.84      0.82       100
           6       0.95      0.96      0.96       100
           7       0.95      0.78      0.86       100
           8       0.73      0.64      0.68       100
           9       0.99      0.98      0.98       100
          10       0.89      0.93      0.91       100
          11       0.89      0.93      0.91       100
          12       0.97      0.83      0.89       100
          13       1.00      0.98      0.99       100
          14       0.96      0.96      0.96       100
          15       0.99      0.96      0.97       100
          16       0.97      0.99      0.98       100
          17       0.96    

# RESULT

k = 200

| Classifier Used | Precision | Recall | F1-score |
| --- | --- | --- | --- |
| Inbuilt MNB classifier | 0.90| 0.90| 0.90|
| My classifier | 0.78 | 0.78 | 0.78 |

k = 500

| Classifier Used | Precision | Recall | F1-score |
| --- | --- | --- | --- |
| Inbuilt MNB classifier | 0.91| 0.91| 0.91|
| My classifier | 0.82 | 0.82 | 0.82 |

k = 1000

| Classifier Used | Precision | Recall | F1-score |
| --- | --- | --- | --- |
| Inbuilt MNB classifier | 0.93| 0.93| 0.93|
| My classifier | 0.90 | 0.90 | 0.90 |

* These values are weighted average